In [37]:
%reload_ext autoreload
%autoreload 2

import torch
import numpy as np
from torch import nn
from matplotlib import pyplot as plt
from models.layers import GaussianLayer
from models.layers import FactorizedEncoder, RecurrentFactorizedEncoder, AutoregressiveEncoder, PriorControlledEncoder
from models.layers import Transition, RecurrentDiscreteDecoder
from models.vate import VATE

[autoreload of models.layers failed: Traceback (most recent call last):
  File "/home/nuzzi/Projects/NeuralEncoder_V2/.venv/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/home/nuzzi/Projects/NeuralEncoder_V2/.venv/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "/home/nuzzi/Projects/NeuralEncoder_V2/.venv/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
  File "/home/nuzzi/Projects/NeuralEncoder_V2/.venv/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 349, in update_class
    if update_generic(old_obj, new_obj):
  File "/home/nuzzi/Projects/NeuralEncoder_V2/.venv/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
  File "/home/nuzzi/Projects/NeuralEncoder_V2/.venv/lib/python3.1

In [38]:
window_len = 50
feature_dim = 32
latent_dim = 3
num_classes = 8

x = torch.zeros((window_len, feature_dim))
y = torch.zeros((window_len, 1), dtype=torch.int32)


In [39]:
encoder = FactorizedEncoder(
    body_module=nn.Sequential(
        nn.LazyLinear(32), nn.GELU(),
        nn.LazyLinear(32), nn.GELU(),
    ),
    gaussian_module=GaussianLayer(output_dim=latent_dim)
    )

z, mu, logvar = encoder.forward(x)

print(z.shape, mu.shape, logvar.shape)

torch.Size([1, 50, 3]) torch.Size([1, 50, 3]) torch.Size([1, 50, 3])


In [40]:
encoder = RecurrentFactorizedEncoder(
    recurrent_module=nn.LSTM(input_size=feature_dim, hidden_size=32, num_layers=1, batch_first=True, bidirectional=False),
    body_module=nn.Sequential(
        nn.LazyLinear(32), nn.GELU(),
        nn.LazyLinear(32), nn.GELU(),
    ),   
    gaussian_module=GaussianLayer(output_dim=latent_dim)
    )

z, mu, logvar = encoder.forward(x)

print(z.shape, mu.shape, logvar.shape)

torch.Size([1, 50, 3]) torch.Size([1, 50, 3]) torch.Size([1, 50, 3])


In [41]:
encoder = AutoregressiveEncoder(
    recurrent_module=nn.LSTM(input_size=feature_dim, hidden_size=32, num_layers=1, batch_first=True, bidirectional=False),
    body_module=nn.Sequential(
        nn.LazyLinear(32), nn.GELU(),
        nn.LazyLinear(32), nn.GELU(),
    ),   
    gaussian_module=GaussianLayer(output_dim=latent_dim)
    )

z, mu, logvar = encoder.forward(x)

print(z.shape, mu.shape, logvar.shape)

torch.Size([1, 50, 3]) torch.Size([1, 50, 3]) torch.Size([1, 50, 3])


In [42]:
encoder = AutoregressiveEncoder(
    recurrent_module=nn.LSTM(input_size=feature_dim, hidden_size=32, num_layers=1, batch_first=True, bidirectional=False),
    body_module=nn.Sequential(
        nn.LazyLinear(32), nn.GELU(),
        nn.LazyLinear(32), nn.GELU(),
    ),   
    gaussian_module=GaussianLayer(output_dim=latent_dim),
    residual_mode='simple'
    )

z, mu, logvar = encoder.forward(x)

print(z.shape, mu.shape, logvar.shape)

torch.Size([1, 50, 3]) torch.Size([1, 50, 3]) torch.Size([1, 50, 3])


In [43]:
encoder = AutoregressiveEncoder(
    recurrent_module=nn.LSTM(input_size=feature_dim, hidden_size=32, num_layers=1, batch_first=True, bidirectional=False),
    body_module=nn.Sequential(
        nn.LazyLinear(32), nn.GELU(),
        nn.LazyLinear(32), nn.GELU(),
    ),   
    gaussian_module=GaussianLayer(output_dim=latent_dim),
    residual_mode='bounded'
    )

z, mu, logvar = encoder.forward(x)

print(z.shape, mu.shape, logvar.shape)

torch.Size([1, 50, 3]) torch.Size([1, 50, 3]) torch.Size([1, 50, 3])


In [44]:
transition = Transition(
    body_module=nn.Sequential(
        nn.LazyLinear(32), nn.GELU(),
        nn.LazyLinear(32), nn.GELU(),
    ),   
    gaussian_module=GaussianLayer(output_dim=latent_dim),
    residual_mode='bounded'
    )

z_next, mu_next, logvar_next = transition.forward(z)

print(z_next.shape, mu_next.shape, logvar_next.shape)

torch.Size([1, 50, 3]) torch.Size([1, 50, 3]) torch.Size([1, 50, 3])


In [45]:
encoder = PriorControlledEncoder(
    recurrent_module=nn.LSTM(input_size=feature_dim, hidden_size=32, num_layers=1, batch_first=True, bidirectional=False),
    body_module=nn.Sequential(
        nn.LazyLinear(32), nn.GELU(),
        nn.LazyLinear(32), nn.GELU(),
    ),   
    transition_module=transition,
    gaussian_module=GaussianLayer(output_dim=latent_dim),
    )

z, mu, logvar = encoder.forward(x)

print(z.shape, mu.shape, logvar.shape)

torch.Size([1, 50, 3]) torch.Size([1, 50, 3]) torch.Size([1, 50, 3])


In [46]:
decoder = RecurrentDiscreteDecoder(
    recurrent_module=nn.LSTM(input_size=latent_dim, hidden_size=32, num_layers=1, batch_first=True, bidirectional=False),
    body_module=nn.Sequential(
        nn.LazyLinear(32), nn.GELU(),
        nn.LazyLinear(32), nn.GELU(),
    ),   
    num_classes=8
)

y_pred = decoder.forward(z)
print(y_pred.shape)

torch.Size([1, 50, 8])


In [61]:
model = VATE(encoder=encoder, transition=transition,
             target_decoders={'test': decoder})

model.summary(input_features=32)

forward_out = model.forward(x)
log_prob_out = model.get_log_probs(forward_out, y)


Module         Parameters     
------------------------------
Encoder        12237          
Transition     1383           
Decoders       
-test          7112           
------------------------------
Total          20732          
